In [55]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict, load_model, Metrics, save_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
process_train_data()
process_test_data()

train_data, test_data = load_split_processed_data()

model_params = {
    "random_state": 42,
    "seed": 42,
    "objective": "binary",
    "metric": "binary_logloss",
    "verbosity": -1,
    "boosting_type": "gbdt",
    "feature_pre_filter": False,
    "lambda_l1": 2.740292203572801,
    "lambda_l2": 9.502614592851208,
    "learning_rate": 0.07586188568634152,
    "num_leaves": 93,
    "feature_fraction": 0.34049127666480095,
    "bagging_fraction": 0.8460348477289328,
    "max_depth": 10,
    "early_stopping_rounds": 88,
}


train_x = train_data.drop(columns="target")
train_y = train_data.target

valid_x = test_data.drop("target", axis=1)[train_x.columns]
valid_y = test_data.target

dtrain = lgb.Dataset(train_x, label=train_y)
dvalid = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)

model = lgb.train(
    model_params,
    dtrain,
    valid_sets=[dvalid],
)

X = test_data[train_x.columns]
y_true = test_data.target

y_pred_proba = model.predict(X, num_iteration=model.best_iteration)
threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)

print("Columns: ", train_x.columns)

2024-06-23 08:44:07 - Running pipeline: TRANSFORM_TRAIN (7:55:41)
2024-06-23 08:44:07 - 0. Running step: set_missings (7:55:41)


Processing train data...
-----------------


2024-06-23 08:44:10 - 1. Running step: reduce_mem_usage (7:55:44)
2024-06-23 08:44:11 - 2. Running step: feature_selection (7:55:45)
2024-06-23 08:44:11 - 3. Running step: merge_train_dpi_features (7:55:45)
2024-06-23 08:44:11 - 4. Running step: merge_train_bnum_features (7:55:45)


Mem. usage decreased to 231.19 Mb (75.0% reduction)


2024-06-23 08:44:11 - 5. Running step: merge_train_fe_features (7:55:45)
2024-06-23 08:44:12 - 6. Running step: merge_train_fe_total_features (7:55:46)
2024-06-23 08:44:12 - 7. Running step: feature_selection_new (7:55:46)
2024-06-23 08:44:12 - 8. Running step: feature_selection_new (7:55:46)
2024-06-23 08:44:12 - 10. Done (7:55:46)
2024-06-23 08:44:12 - Running pipeline: TRANSFORM_TEST (7:55:46)
2024-06-23 08:44:12 - 0. Running step: set_missings (7:55:46)


-----------------

Shape: (149998, 50)
Columns: Index(['sms_in_cnt_std_total', 'casual_act_count', 'loc_market_share',
       'imei_mean_days_usage', 'SUM_of_Count_events_susp_app_sum',
       'voice_mts_in_dwork_part_mea_mnt3', 'num_act_days_std_total',
       'voice_out_td_cnt_std_total', 'MV_dou_net',
       'non_accum_internet_vol_td_total', 'voice_in_td_cnt_td_mnt3',
       'MAX_of_day_cnt_susp_app_sum', 'data_3g_tv_cnt_std_mnt3',
       'content_cnt_mea_diff', 'days_of_end_last_ppm',
       'num_act_days_min_total', 'num_act_days_mea_mnt3', 'Balance_uah',
       'sum_paym_6_month', 'day_end_gba', 'sms_in_cnt_max_total',
       'voice_in_tar_dur_min_mnt1', 'voice_mts_in_dwork_part_mea_total',
       'other_cnt_sms_in_sum', 'voice_in_tar_dur_min_total',
       'num_act_days_td_total', 'all_cnt_std_mnt3', 'all_cnt_mea_diff',
       'non_accum_internet_vol_mea_diff', 'sms_in_cnt_mea_total',
       'casual_topic_cnt_sms_in_sum', 'num_act_days_min_mnt3',
       'num_act_days_std_mnt3',

2024-06-23 08:44:15 - 1. Running step: reduce_mem_usage (7:55:49)
2024-06-23 08:44:16 - 2. Running step: feature_selection (7:55:50)
2024-06-23 08:44:16 - 3. Running step: merge_test_dpi_features (7:55:50)
2024-06-23 08:44:16 - 4. Running step: merge_test_bnum_features (7:55:50)


Mem. usage decreased to 225.88 Mb (75.5% reduction)


2024-06-23 08:44:16 - 5. Running step: merge_test_fe_features (7:55:50)
2024-06-23 08:44:17 - 6. Running step: merge_test_fe_total_features (7:55:50)
2024-06-23 08:44:17 - 8. Done (7:55:51)


-----------------

Shape: (150000, 951)
Columns: Index(['num_act_days_mea_wk1', 'device_brand_other', 'num_act_days_min_mnt1',
       'active_ppm', 'num_act_days_min_mnt3', 'num_act_days_mea_mnt1',
       'loc_market_share', 'voice_mts_in_dwork_part_min_mnt1',
       'voice_in_kievstar_part_std_mnt1', 'device_type_smartphone',
       ...
       'accum_oth_dur_td_total', 'data_3g_tv_cnt_td_total',
       'voice_in_td_cnt_min_total', 'abon_part_td_total',
       'com_num_part_std_total', 'sms_in_cnt_std_total',
       'pay_p2p_out_sum_td_total', 'pay_max_td_total',
       'voice_in_short_part_td_total', 'voice_out_cmpttrs_avg_dumin_total'],
      dtype='object', length=951)
Data saved

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.71      0.32      0.44      9403

    accuracy                           0.95    150000
   macro avg       0.83      0.66      0.71    1

In [54]:
save_model(model, "LightGbmV2_new_total_features_p072_r032_090auc", list(train_x.columns))

Save LightGbmV2_new_total_features_p072_r032_090auc


'LightGbmV2_new_total_features_p072_r032_090auc'